In [1]:
%pip install opfython pandas numpy scikit-learn matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------- ----------- 5.8/8.1 MB 31.2 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 32.9 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 42.0 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 40.2 MB/s  0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ---------------------------------------- 0/7 [pyparsing]
   ---------------------------------------- 0/7 [pyparsing]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
   ----- -------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [153]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import TargetEncoder,LabelEncoder,OneHotEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from opfython.models import SupervisedOPF
import opfython.math.general as opf_g
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,ConfusionMatrixDisplay
import time
import matplotlib.pyplot as plt
import os
np.seterr(all='ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

##ANALISE DO KDD-CUP

In [154]:
from sklearn.datasets import fetch_kddcup99

data = fetch_kddcup99(as_frame=True, percent10=True)
df = data.frame



duplicates = df.duplicated()
print(duplicates.sum(), len(df), len(df)-duplicates.sum())

df.drop_duplicates(inplace=True)
df.head()


348435 494021 145586


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,b'normal.'


In [155]:
df["protocol_type"].value_counts()


protocol_type
b'tcp'     130913
b'udp'      12267
b'icmp'      2406
Name: count, dtype: int64

In [156]:
df["service"].value_counts()


service
b'http'        62054
b'private'     49057
b'smtp'         9721
b'domain_u'     5425
b'other'        4769
               ...  
b'X11'            11
b'tim_i'           5
b'pm_dump'         1
b'tftp_u'          1
b'red_i'           1
Name: count, Length: 66, dtype: int64

In [157]:
df["flag"].value_counts()

flag
b'SF'        87459
b'S0'        42278
b'REJ'       14712
b'RSTO'        569
b'RSTR'        425
b'S1'           57
b'SH'           34
b'S2'           24
b'RSTOS0'       11
b'S3'           10
b'OTH'           7
Name: count, dtype: int64

In [158]:
df["land"].value_counts()

land
0    145566
1        20
Name: count, dtype: int64

In [159]:
df["logged_in"].value_counts()

logged_in
0    74032
1    71554
Name: count, dtype: int64

In [160]:
df["is_host_login"].value_counts()

is_host_login
0    145586
Name: count, dtype: int64

In [161]:
df["is_guest_login"].value_counts()

is_guest_login
0    144901
1       685
Name: count, dtype: int64

##PRÉ PROCESSAMENTO KDD-CUP

O dataset KDD-CUP consiste em dados de pacotes de dados que podem ou não representar ataques a uma rede.

A versão utilizada no artigo foi uma versão reduzida, com 10% das amostras. Cada amostra possui 41 features além de um rótulo que pode pertencer a 23 classes (22 representam ataques e 1 representa pacotes normais).

Conforme o texto, foram utilizadas as seguintes variantes do conjunto de dados:

*   KddCup-5:  6 classes (back DoS,buffer overflow, ftp write, guess passwd, imap e normal);
*   KddCup-10: 11 classes (ipsweep, land DoS, loadmodule, multihop, neptune, além das classes em KddCup-5)
*   KddCup-15: 16 classes (nmap, perl, phf, pod DoS, portsweep, além das classes em KddCup-10)

As features categóricas foram tratadas com One-Hot Encoder e todas as features foram padronizadas com StandardScaler.

In [162]:
kddCup_5_classes = [b"back.",b"ftp_write.",b"buffer_overflow.",b"guess_passwd.",b"imap.",b"normal."]
kddCup5_mask = df["labels"].isin(kddCup_5_classes)
kddCup5 = df[kddCup5_mask]
kddCup5["labels"].value_counts()

labels
b'normal.'             87832
b'back.'                 968
b'guess_passwd.'          53
b'buffer_overflow.'       30
b'imap.'                  12
b'ftp_write.'              8
Name: count, dtype: int64

In [163]:
kddCup_10_classes = [
    b"back.",
    b"ftp_write.",
    b"buffer_overflow.",
    b"guess_passwd.",
    b"imap.",
    b"ipsweep.",
    b"land.",
    b"loadmodule.",
    b"neptune.",
    b"multihop.",
    b"normal."
]

kddCup10_mask = df["labels"].isin(kddCup_10_classes)
kddCup10 = df[kddCup10_mask]
kddCup10["labels"].value_counts()

labels
b'normal.'             87832
b'neptune.'            51820
b'back.'                 968
b'ipsweep.'              651
b'guess_passwd.'          53
b'buffer_overflow.'       30
b'land.'                  19
b'imap.'                  12
b'loadmodule.'             9
b'ftp_write.'              8
b'multihop.'               7
Name: count, dtype: int64

In [164]:
kddCup_15_classes = [b"back.",b"ftp_write.",b"buffer_overflow.",b"guess_passwd.",b"imap.",b"ipsweep.",b"land.",b"loadmodule.",b"neptune.",b"multihop.",b"normal.",b"nmap.",b"perl.",b"phf.",b"pod.",b"portsweep."]
kddCup15_mask = df["labels"].isin(kddCup_15_classes)
kddCup15 = df[kddCup15_mask]
kddCup15["labels"].value_counts()

labels
b'normal.'             87832
b'neptune.'            51820
b'back.'                 968
b'ipsweep.'              651
b'portsweep.'            416
b'pod.'                  206
b'nmap.'                 158
b'guess_passwd.'          53
b'buffer_overflow.'       30
b'land.'                  19
b'imap.'                  12
b'loadmodule.'             9
b'ftp_write.'              8
b'multihop.'               7
b'phf.'                    4
b'perl.'                   3
Name: count, dtype: int64

Em todas as variantes, percebe-se o imenso desbalanceamento entre as classes. No entanto, o artigo não busca formas de tratar esse desbalanceamento. Ao invés disso, utiliza métricas que levam em consideração o desbalanceamento das classes para avaliar o modelo.

##TREINAMENTO KDD-CUP

In [165]:
def kdd_preprocessing(dataset):
  """ 
  Pre-processamento do dataset kddcup
  """
  X = dataset.iloc[:,:-1]
  y = dataset.iloc[:,-1]

  X = pd.get_dummies(X, columns=["service","protocol_type", "flag"], dtype=float)
  label_encoder = LabelEncoder()
  y = pd.Series(label_encoder.fit_transform(y))

  return X,y,label_encoder

In [166]:
def cria_folds(X):
  """ 
  Separa os folds de treino e teste. Ao total serao 20 folds. 10 para treino, 10 para teste 
  """
  kfold = KFold(n_splits=20)

  folds = list(kfold.split(X))

  rng = np.random.default_rng(42)  # random para garantir repetibilidade

  # escolher 10 folds aleatórios para treino
  train_folds_idx = rng.choice(20, size=10, replace=False)
  # os restantes são teste 
  test_folds_idx = [i for i in range(20) if i not in train_folds_idx]


  return train_folds_idx,test_folds_idx,folds

In [ ]:
def cross_validation_knn(X,y,params):
  """Recebe um fold de treino e os parâmetros a serem testados. Retorna os parametros com melhor media de opf-style accuracy"""
  SKFold = KFold(n_splits=5)

  K_avg_accs = []
  
  max_label_global = np.max(y)

  for k in params["K"]:
    K_accs = []

    for i, (train_idxs,val_idxs) in enumerate(SKFold.split(X,y)):
      print(f"K = {k}, fold {i}")

      X_train, X_val = X[train_idxs],X[val_idxs]
      y_train, y_val = y[train_idxs],y[val_idxs]

      
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(X_train,y_train)
      predicts = knn.predict(X_val)

      #ajuste necessario pois o fold de validacao pode contar com menos classes do que os de treino.
      #isso pode acontecer porque ha classes com poucas amostras
      y_val_calc = y_val
      predicts_calc = predicts
      
      if np.max(y_val) < max_label_global:
          y_val_calc = np.append(y_val, max_label_global)
          predicts_calc = np.append(predicts, max_label_global)

      opf_acc = opf_g.opf_accuracy(y_val_calc, predicts_calc)
      #print(f"opf_acc = {opf_acc}")
      K_accs.append(opf_acc)

    mean = np.array(K_accs).mean()
    print(f"media para k={k}: {mean}")

    K_avg_accs.append(mean)
  
  melhor_K = np.array(K_avg_accs).argmax()
  print(f"Melhor k = {params['K'][melhor_K]}")
  
  return params["K"][melhor_K] 

In [ ]:
def cross_validation_svm(X, y, params):
    """Recebe um fold de treino e os parâmetros a serem testados. Retorna os parametros com melhor media de opf-style accuracy"""
    SKFold = KFold(n_splits=5)

    # Matriz de médias: linhas = Cs, colunas = gammas
    C_gamma_avg_accs = np.zeros((len(params["C"]), len(params["gamma"])))
    
    max_label_global = np.max(y)
    for i_c, C_value in enumerate(params["C"]):
        for j_g, gamma_value in enumerate(params["gamma"]):

            gamma_accs = []

            for fold, (train_idx, val_idx) in enumerate(SKFold.split(X, y)):
                X_train, X_val = X[train_idx], X[val_idx]
                y_train, y_val = y[train_idx], y[val_idx]

                svm = SVC(kernel="rbf", gamma=gamma_value, C=C_value)
                svm.fit(X_train, y_train)
                predicts = svm.predict(X_val)
                
                #ajuste necessario pois o fold de validacao pode contar com menos classes do que os de treino.
                #isso pode acontecer porque ha classes com poucas amostras
                y_val_calc = y_val
                predicts_calc = predicts
                
                if np.max(y_val) < max_label_global:
                    y_val_calc = np.append(y_val, max_label_global)
                    predicts_calc = np.append(predicts, max_label_global)

                opf_acc = opf_g.opf_accuracy(y_val_calc, predicts_calc)
                
                gamma_accs.append(opf_acc)

            C_gamma_avg_accs[i_c, j_g] = np.mean(gamma_accs)
            print(f"C={C_value}, gamma={gamma_value}, média = {C_gamma_avg_accs[i_c, j_g]}")

    # encontra o melhor índice (C, gamma)
    best_idx = np.unravel_index(np.argmax(C_gamma_avg_accs), C_gamma_avg_accs.shape)
    best_C = params["C"][best_idx[0]]
    best_gamma = params["gamma"][best_idx[1]]

    print(f"\nMelhor combinação: C={best_C}, gamma={best_gamma}")
    return best_C, best_gamma


In [ ]:
def treinar(modelo,nome_do_modelo,X_train,y_train):
    """Treina o modelo e conta o tempo que levou"""
    tempo_inicial = time.perf_counter()
    modelo.fit(X_train,y_train)
    tempo_final = time.perf_counter()
    tempo_decorrido = tempo_final - tempo_inicial
    print(f"Tempo de treino {nome_do_modelo}: {tempo_decorrido:3f}s")
    return tempo_decorrido

In [ ]:
def testar(modelo,nome_do_modelo,X_test):
    """Testa o modelo e conta o tempo que levou"""
    tempo_inicial = time.perf_counter()
    predicts = modelo.predict(X_test)
    tempo_final = time.perf_counter()
    tempo_decorrido = tempo_final - tempo_inicial
    print(f"Tempo de treino {nome_do_modelo}: {tempo_decorrido:3f}s")
    return tempo_decorrido,predicts

In [171]:
def matriz_de_confusao(cm, display_labels, dir, nome):
    os.makedirs(dir, exist_ok=True)

    n_classes = cm.shape[0]
    fig_size = max(6, n_classes * 0.7)

    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
    disp.plot(ax=ax, xticks_rotation='vertical', colorbar=True)

    ax.set_title(nome)

    plt.tight_layout()
    plt.savefig(f"{dir}/{nome}.png", dpi=300)
    plt.close()

In [ ]:

def kdd_experiment(dataset,dataset_name):

  #inicializacao dos parametros de grid search
  knn_cv_params = {"K":[3,5,7,9,11]}

  svm_cv_params = {"gamma":[0.001, 0.01, 0.1],
                   "C":[0.1, 1, 10]
                   }
  
  #inicializacao dos vetores de metricas
  all_acc_opf_opf = np.array([])
  all_acc_opf = np.array([])
  all_tempo_treino_opf = np.array([])
  all_tempo_teste_opf = np.array([])
  all_f1_opf = np.array([])

  all_acc_opf_svm = np.array([])
  all_acc_svm = np.array([])
  all_tempo_treino_svm = np.array([])
  all_tempo_teste_svm = np.array([])
  all_f1_svm = np.array([])

  all_acc_opf_knn =np.array([])
  all_acc_knn = np.array([])
  all_tempo_treino_knn = np.array([])
  all_tempo_teste_knn = np.array([])
  all_f1_knn = np.array([])


  #shuffle do dataset
  dataset = dataset.sample(frac=1, random_state=42)
  
  X,y,y_encoder = kdd_preprocessing(dataset)
  
  #variaveis auxiliares para as matrizes de confusao
  labels = np.unique(y)
  max_classes = len(labels)
  classes_names = np.unique(y)
  classes_names = y_encoder.inverse_transform(classes_names)
  

  train_folds_idx, test_folds_idx,folds = cria_folds(X)

  #matrizes de confusao
  cm_opf = np.zeros((max_classes, max_classes), dtype=int)
  cm_svm = np.zeros((max_classes, max_classes), dtype=int)
  cm_knn = np.zeros((max_classes, max_classes), dtype=int)

 
  #a cada run, os modelos sao treinados com um fold e testado com um fold (2*10 = 20)
  for run in range(10):
    print(f"\nExecução {run+1}")

    # selecao dos folds
    train_idx = folds[train_folds_idx[run]][1]
    test_idx  = folds[test_folds_idx[run]][1]
      
    X_train = X.iloc[train_idx].copy()
    y_train = y.iloc[train_idx].copy()
    X_test  = X.iloc[test_idx].copy()
    y_test  = y.iloc[test_idx].copy()
    
    #print(f"Tamanho treino: {len(X_train)}\nTamanho Teste:{len(X_test)}")
    X_train_np = X_train.values
    y_train_np = y_train.values
    X_test_np = X_test.values
    y_test_np = y_test.values

    #padronizacao dos dados
    standard_scaler = StandardScaler()
    X_train_np = standard_scaler.fit_transform(X_train_np)
    X_test_np = standard_scaler.transform(X_test_np)

    #pca = PCA(n_components=10)
    #X_train_np = pca.fit_transform(X_train_np)
    #X_test_np = pca.transform(X_test_np)

    opf = SupervisedOPF(distance="log_squared_euclidean", pre_computed_distance=None)

    K = cross_validation_knn(X_train_np,y_train_np,knn_cv_params)
    knn = KNeighborsClassifier(n_neighbors=K)

    C,gamma = cross_validation_svm(X_train_np,y_train_np,svm_cv_params)
    svm = SVC(kernel='rbf',C=C,gamma=gamma)
    
    
    t_train_opf = treinar(opf, "OPF", X_train_np, y_train_np)
    t_train_svm = treinar(svm, "SVM", X_train_np, y_train_np)
    t_train_knn = treinar(knn, "KNN", X_train_np, y_train_np)

    t_test_opf,preds_opf = testar(opf, "OPF", X_test_np)
    t_test_svm,preds_svm = testar(svm, "SVM", X_test_np)
    t_test_knn,preds_knn = testar(knn, "KNN", X_test_np)

    acc_opf_opf = opf_g.opf_accuracy(y_test_np, preds_opf)
    acc_opf = accuracy_score(y_test_np, preds_opf)
    f1_opf = f1_score(y_test_np, preds_opf,average='macro')
  
    acc_opf_svm = opf_g.opf_accuracy(y_test_np, preds_svm)
    acc_svm = accuracy_score(y_test_np, preds_svm)
    f1_svm = f1_score(y_test_np, preds_svm,average='macro')

    acc_opf_knn = opf_g.opf_accuracy(y_test_np, preds_knn)
    acc_knn = accuracy_score(y_test_np, preds_knn)
    f1_knn = f1_score(y_test_np, preds_knn,average='macro')

    all_acc_opf_opf = np.append(all_acc_opf_opf, acc_opf_opf)
    all_acc_opf = np.append(all_acc_opf, acc_opf)
    all_tempo_treino_opf = np.append(all_tempo_treino_opf, t_train_opf)
    all_tempo_teste_opf = np.append(all_tempo_teste_opf, t_test_opf)
    all_f1_opf = np.append(all_f1_opf,f1_opf)

    all_acc_opf_svm = np.append(all_acc_opf_svm, acc_opf_svm)
    all_acc_svm = np.append(all_acc_svm, acc_svm)
    all_tempo_treino_svm = np.append(all_tempo_treino_svm, t_train_svm)
    all_tempo_teste_svm = np.append(all_tempo_teste_svm, t_test_svm)
    all_f1_svm = np.append(all_f1_svm,f1_svm)

    all_acc_opf_knn = np.append(all_acc_opf_knn, acc_opf_knn)
    all_acc_knn = np.append(all_acc_knn, acc_knn)
    all_tempo_treino_knn = np.append(all_tempo_treino_knn, t_train_knn)
    all_tempo_teste_knn = np.append(all_tempo_teste_knn, t_test_knn)
    all_f1_knn = np.append(all_f1_knn,f1_knn)
    
    cm_svm += confusion_matrix(y_test_np,preds_svm,labels=labels)
    cm_opf += confusion_matrix(y_test_np,preds_opf,labels=labels)
    cm_knn += confusion_matrix(y_test_np,preds_knn,labels=labels)

  results = pd.DataFrame({
    "Modelos": ["OPF", "SVM", "KNN"],

    "Acc OPF-Style (mean)": [
        all_acc_opf_opf.mean(),
        all_acc_opf_svm.mean(),
        all_acc_opf_knn.mean()
    ],
    "Acc OPF-Style (std)": [
        all_acc_opf_opf.std(),
        all_acc_opf_svm.std(),
        all_acc_opf_knn.std()
    ],

    "Accuracy (mean)": [
        all_acc_opf.mean(),
        all_acc_svm.mean(),
        all_acc_knn.mean()
    ],
    "Accuracy (std)": [
        all_acc_opf.std(),
        all_acc_svm.std(),
        all_acc_knn.std()
    ],

    "F1 score (mean)": [
        all_f1_opf.mean(),
        all_f1_svm.mean(),
        all_f1_knn.mean()
    ],
    "F1 score (std)": [
        all_f1_opf.std(),
        all_f1_svm.std(),
        all_f1_knn.std()
    ],

    "Tempo Treino (mean)": [
        all_tempo_treino_opf.mean(),
        all_tempo_treino_svm.mean(),
        all_tempo_treino_knn.mean()
    ],
    "Tempo Treino (std)": [
        all_tempo_treino_opf.std(),
        all_tempo_treino_svm.std(),
        all_tempo_treino_knn.std()
    ],

    "Tempo Teste (mean)": [
        all_tempo_teste_opf.mean(),
        all_tempo_teste_svm.mean(),
        all_tempo_teste_knn.mean()
    ],
    "Tempo Teste (std)": [
        all_tempo_teste_opf.std(),
        all_tempo_teste_svm.std(),
        all_tempo_teste_knn.std()
    ]
    })
  
  matriz_de_confusao(cm_svm,classes_names,f"{dataset_name}/","SVM")
  matriz_de_confusao(cm_opf,classes_names,f"{dataset_name}/","OPF")
  matriz_de_confusao(cm_knn,classes_names,f"{dataset_name}/","KNN")
  results.to_csv(f"{dataset_name}/{dataset_name}.csv")


kdd_experiment(kddCup5,"KDDCUP-5")
kdd_experiment(kddCup10,"KDDCUP-10")
kdd_experiment(kddCup15,"KDDCUP-15")

(6, 6)

Execução 1
4446 4446
The arrays do not have common elements.
Tamanho treino: 4446
Tamanho Teste:4446
2025-11-28 18:36:38,748 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2025-11-28 18:36:38,749 - opfython.core.opf — INFO — Creating class: OPF.
2025-11-28 18:36:38,750 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2025-11-28 18:36:38,750 - opfython.core.opf — INFO — Class created.
2025-11-28 18:36:38,750 - opfython.models.supervised — INFO — Class overrided.
K = 3, fold 0
K = 3, fold 1
K = 3, fold 2
K = 3, fold 3
K = 3, fold 4
media para k=3: 0.9347715241356402
K = 5, fold 0
K = 5, fold 1
K = 5, fold 2
K = 5, fold 3
K = 5, fold 4
media para k=5: 0.9354088238746765
K = 7, fold 0
K = 7, fold 1
K = 7, fold 2
K = 7, fold 3
K = 7, fold 4
media para k=7: 0.8634957479405092
K = 9, fold 0
K = 9, fold 1
K = 9, fold 2
K = 9, fold 3
K = 9, fold 4
media para k=9: 0.8634957479405092
K = 11, fold 0
K = 11, fold 1
